<a href="https://colab.research.google.com/github/GerardoMunoz/embedded/blob/main/WiFi_Sockets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
┌───────────────────────────────┐
│ Application Layer             │   ← Your code: HTTP, MQTT, CoAP, WebSocket, others
├───────────────────────────────┤
│ Transport Layer               │   ← TCP (reliable) or UDP (lightweight), others
├───────────────────────────────┤
│ Internet / Network Layer       │   ← IP (Internet Protocol), others
├───────────────────────────────┤
│ Link / Access Layer            │   ← Wi-Fi, Ethernet, Cellular, LoRa, BLE, others
├───────────────────────────────┤
│ Physical Layer                 │   ← cables, Radio waves: local, cellphones, satellite
└───────────────────────────────┘

## MicroPython network Library - WiFi Connection (Raspberry Pi Pico W)

In [ ]:
from machine import Pin
import network, time
ap_mode_pin=16

# Cambia True/False según quieras AP o estación
ap_mode = Pin(ap_mode_pin, mode=Pin.IN, pull=Pin.PULL_UP) # Example using GPIO pin 16


if bool(ap_mode.value()) :
    wlan = network.WLAN(network.AP_IF)       # Interface punto de acceso
    wlan.active(True)
    wlan.config(essid="RPi-Pico", password="12345678")
    if wlan.active():
        print("AP activo")
        print("SSID:", wlan.config('essid'))
        print("IP Address:", wlan.ifconfig()[0])
    else:
        print("No se pudo activar el AP, estado:", wlan.status())

else:
    wlan = network.WLAN(network.STA_IF)      # Interface estación
    wlan.active(True)
    wlan.connect("Ejemplo", "12345678")
    for _ in range(10):
        if wlan.isconnected():
            break
        print('.', end='')
        time.sleep(1)
    if wlan.isconnected():
        print("\nConectado. IP:", wlan.ifconfig()[0])
    else:
        print("\nError de conexión. Estado:", wlan.status())
            # Handle connection error
            # Error meanings
            # 0  Link Down
            # 1  Link Join
            # 2  Link NoIp
            # 3  Link Up
            # -1 Link Fail
            # -2 Link NoNet
            # -3 Link BadAuth







The `network` module in MicroPython is used to configure the Wi-Fi interface on the Raspberry Pi Pico W.



Method / Property   Description

* WLAN(mode)  Create a WLAN object.
    * Use network.STA_IF for station mode.
    * Use network.AP_IF for access point mode.
* active([is_active]) Get or set whether the interface is active.
* connect(ssid, password) Connect to a WiFi network.
* isconnected()   Returns True if connected to a WiFi network.
* disconnect()    Disconnect from the network.
* ifconfig()  Get/set IP configuration: (ip, subnet, gateway, dns)
* status()    Returns the status code of the WiFi connection.
* scan()  Returns a list of available WiFi networks.
* config('param') Get or set configuration parameters like MAC address.



## MicroPython socket Library - Networking on Raspberry Pi Pico W

The `socket` module allows for low-level network communication using standard socket interfaces (like in CPython), with support for TCP and UDP protocols.




### Common TCP Use Cases


1. TCP Client (e.g., connect to a web server)
```python
import socket

addr = socket.getaddrinfo("example.com", 80)[0][-1]

s = socket.socket()
s.connect(addr)
s.send(b"GET / HTTP/1.0\r\nHost: example.com\r\n\r\n")

# Receive response
while True:
    data = s.recv(100)
    if data:
        print(data.decode(), end="")
    else:
        break

s.close()
```



2.  TCP Server (e.g., basic web server)
```python
import socket

addr = socket.getaddrinfo('0.0.0.0', 80)[0][-1]

s = socket.socket()
s.bind(addr)
s.listen(1)

print("Listening on", addr)

while True:
    cl, addr = s.accept()
    print("Client connected from", addr)
    request = cl.recv(1024)
    print("Request:", request)

    response = b"""\
HTTP/1.0 200 OK\r
Content-Type: text/html\r\n\r
<html><body><h1>Hello from Pico W</h1></body></html>
"""
    cl.send(response)
    cl.close()
```







3. socket Methods & Constants

Method / Constant	Description

* socket()	Create a new socket object.
* connect((host, port))	Connect to remote server (TCP).
* bind((host, port))	Bind to local address (for servers).
* listen([backlog])	Start listening for incoming connections.
* accept()	Accept a new client connection.
* recv(bufsize)	Receive data (TCP).
* recvfrom(bufsize)	Receive data (UDP).
* send(data)	Send data (TCP).
* sendto(data, addr)	Send data (UDP).
* close()	Close the socket.
* settimeout(seconds)	Set a timeout for blocking operations.
* AF_INET	Address family for IPv4.
* SOCK_STREAM	Socket type for TCP.
* SOCK_DGRAM	Socket type for UDP.





### 4. Pub/Sub Example
#### A. Minimal Broker Skeleton

In [ ]:
# broker.py
import network, socket, ujson, time

SSID = "yourwifi"
PWD  = "yourpass"
PORT = 5051

# Wi-Fi
w = network.WLAN(network.STA_IF)
w.active(True)
w.connect(SSID, PWD)
while w.status() != 3: time.sleep(0.5)
print("IP:", w.ifconfig()[0])

# TCP socket
srv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
srv.bind(('', PORT))
srv.listen(5)
srv.setblocking(False)

clients = []          # all sockets
subscriptions = {}    # topic -> [sockets]

def send(sock, obj):
    try:
        sock.send(ujson.dumps(obj).encode() + b"\n")
    except:
        if sock in clients: clients.remove(sock)
        for lst in subscriptions.values():
            if sock in lst: lst.remove(sock)
        sock.close()

print("Broker listening")

while True:
    # Accept new
    try:
        c, a = srv.accept()
        c.setblocking(False)
        clients.append(c)
        print("Client:", a)
    except OSError:
        pass

    # Check messages
    for c in clients[:]:
        try:
            raw = c.recv(1024)
            if raw:
                for line in raw.split(b"\n"):
                    if not line: continue
                    msg = ujson.loads(line)
                    action = msg.get("action")
                    topic  = msg.get("topic")
                    if action == "SUB":
                        subscriptions.setdefault(topic, []).append(c)
                    elif action == "PUB":
                        for s in subscriptions.get(topic, []):
                            if s != c:
                                send(s, {"topic": topic, "data": msg.get("data")})
            else:
                clients.remove(c)
                c.close()
        except OSError:
            pass
    time.sleep(0.05)


#### B. Simple Client

In [ ]:
# client.py
import network, socket, ujson, _thread, time, sys

SSID = "yourwifi"
PWD  = "yourpass"
BROKER = "192.168.1.50"
PORT   = 5051

# Wi-Fi
w = network.WLAN(network.STA_IF)
w.active(True)
w.connect(SSID, PWD)
while w.status() != 3: time.sleep(0.5)
print("IP:", w.ifconfig()[0])

# TCP
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((BROKER, PORT))

def listen():
    while True:
        try:
            data = s.recv(1024)
            for line in data.split(b"\n"):
                if line:
                    msg = ujson.loads(line)
                    print("[{}] {}".format(msg["topic"], msg["data"]))
        except:
            pass
        time.sleep(0.05)

_thread.start_new_thread(listen, ())

# subscribe
s.send(ujson.dumps({"action": "SUB", "topic": "chat/general"}).encode() + b"\n")

while True:
    txt = input(">> ")
    pkt = {"action": "PUB", "topic": "chat/general", "data": txt}
    s.send(ujson.dumps(pkt).encode() + b"\n")


#### C. Instructions

1. Flash and Prepare

Install latest MicroPython firmware for Pico W (download UF2 from micropython.org
 and drag-drop to the Pico’s USB mass-storage).

Install Thonny or use mpremote so you can copy .py files and run them.

2. Broker (the “server”)

On the Pico you want to act as the broker, copy the file broker.py from my snippet.

Edit the Wi-Fi details:

SSID = "yourwifi"
PWD  = "yourpassword"
PORT = 5051


Save and Run the script in Thonny.

You’ll see something like:

IP: 192.168.1.50
Broker listening


Write down the IP, that’s where clients connect.

This Pico now waits for TCP connections and keeps track of who subscribed to which topic.

3. Client

On another Pico W (or even the same, in a new Thonny window), copy client.py.

Change:

BROKER = "192.168.1.50"   # the IP you saw from the broker
SSID   = "yourwifi"
PWD    = "yourpassword"


Save and run. The client automatically sends:

{"action": "SUB", "topic": "chat/general"}


which subscribes it to that topic.

You’ll see a prompt:

>>


Type something, press Enter → it publishes JSON back to the broker, which then forwards to every other client subscribed to the same topic.

4. Run Multiple Clients

Flash client.py onto a second Pico (or open two Thonny shells).

Both subscribe to chat/general.

When one types a message, the other prints:

[chat/general] hello world

5. Key Details
|Piece |	Purpose|
|-|-|
|broker.py |	Accepts TCP sockets, keeps a subscriptions dict, forwards PUB messages|
|client.py |	Connects to broker, SUBscribes, PUBlishes typed text |
|Topic |	Simple string, e.g. "chat/general" |
|Payload |	Anything JSON-serializable (str, numbers, nested dicts, etc.) |
|Delimiter |	Broker & client split messages by newline \n |

6. Stopping / Restarting

Press Ctrl-C in Thonny to stop.

If Wi-Fi disconnects, just re-run; the scripts reconnect on next run.

Notes & Tips

It’s plain TCP, no real MQTT features (retain, QoS). But perfect to see pub/sub mechanics.

For multiple topics, just send different "topic": "sensor/temp" values.

If you need concurrency that’s smoother, the same logic can be written with uasyncio instead of polling loops.